# Metro Manila Flood Level Predictor

## Importing libraries and dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('AEGISDataset.csv')
data.head()

,lat,lon,flood_heig,elevation,precipitat
0,14.640394,121.055708,0,54.553295,9
1,14.698299,121.002132,0,21.856272,10
2,14.698858,121.100261,0,69.322807,16
3,14.571310,120.983334,0,10.987241,8
4,14.762232,121.075735,0,87.889847,18


## Preparing dataset for training and testing

In [ ]:
x = data.drop('flood_heig', axis=1)
y = data['flood_heig']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y,train_size=0.7)